In [1]:
import os
import xarray as xr
import geopandas as gpd  # type: ignore

from QueryHandler import QueryHandler
from dotenv import load_dotenv
import sqlalchemy as sq
import sys
import pandas as pd
import matplotlib.pyplot as plt  # type: ignore

sys.path.append("../")
from Shared.DataService import DataService

/home/student/dodhiajk/CGC_Grain_Outcome_Predictions/.venv/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_105731/3485261827.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. 

In [3]:
def updateLog(fileName: str, message: str) -> None:
    try:
        if fileName is not None:
            with open(fileName, "a") as log:
                log.write(message + "\n")
    except Exception as e:
        print(message)

In [ ]:
LOG_FILE = "/data/pull_moisture.log"

load_dotenv()
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")

In [4]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    updateLog(LOG_FILE, "Missing database credentials")
    raise ValueError("Environment variables are not set")
else:
    # connicting to database
    db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
    conn = db.connect()

In [5]:
query = sq.text("select * FROM public.soil_moisture")
sm_df = pd.read_sql(query, conn)

In [6]:
sm_df = (
    sm_df.groupby(["date", "cr_num", "car_uid"])
    .agg({"soil_moisture": ["min", "max", "mean"]})
    .reset_index()
)
sm_df.columns = [  # type: ignore
    "date",
    "cr_num",
    "district",
    "soil_moisture_min",
    "soil_moisture_max",
    "soil_moisture_mean",
]

,id,lon,lat,date,cr_num,car_uid,soil_moisture
0,1,-117.625,59.875,1999-10-29,10,4870,0.229969
1,2,-117.375,59.875,1999-10-29,10,4870,0.277028
2,3,-117.125,59.875,1999-10-29,10,4870,0.253480
3,4,-116.875,59.875,1999-10-29,10,4870,0.201568
4,5,-116.625,59.875,1999-10-29,10,4870,0.216237
...,...,...,...,...,...,...,...
5240828,5240829,-98.375,49.125,1993-07-24,3,4608,0.205054
5240829,5240830,-98.125,49.125,1993-07-24,3,4608,0.185292
5240830,5240831,-97.875,49.125,1993-07-24,3,4608,0.173992
5240831,5240832,-97.625,49.125,1993-07-24,3,4608,0.194576


In [8]:
sm_df["date"] = pd.to_datetime(sm_df["date"])
sm_df["day"] = sm_df["date"].dt.day
sm_df["month"] = sm_df["date"].dt.month
sm_df["year"] = sm_df["date"].dt.year

In [9]:
sm_df.drop(columns="date", inplace=True)

In [ ]:
sm_df = sm_df.reindex(
    columns=[
        "year",
        "month",
        "day",
        "cr_num",
        "district",
        "soil_moisture_min",
        "soil_moisture_min",
        "soil_moisture_min",
    ]
)

In [ ]:
sm_df["date"] = pd.to_datetime(sm_df["date"])
sm_df["day"] = sm_df["date"].dt.day
sm_df["month"] = sm_df["date"].dt.month
sm_df["year"] = sm_df["date"].dt.year

In [14]:
sm_df.drop(columns="date", inplace=True)

In [16]:
sm_df = sm_df.reindex(
    columns=[
        "year",
        "month",
        "day",
        "cr_num",
        "district",
        "soil_moisture_min",
        "soil_moisture_min",
        "soil_moisture_min",
    ]
)

In [17]:
sm_df

,year,month,day,cr_num,district,soil_moisture_min,soil_moisture_min,soil_moisture_min
0,1978,11,1,0,4612,0.165900,0.165900,0.165900
1,1978,11,1,5,4740,0.127140,0.127140,0.127140
2,1978,11,1,5,4741,0.153398,0.153398,0.153398
3,1978,11,1,7,4770,0.140772,0.140772,0.140772
4,1978,11,1,7,4771,0.116810,0.116810,0.116810
...,...,...,...,...,...,...,...,...
117216,2017,12,19,10,4870,0.266332,0.266332,0.266332
117217,2017,12,22,5,4732,0.191398,0.191398,0.191398
117218,2017,12,22,7,4790,0.211194,0.211194,0.211194
117219,2017,12,23,7,4790,0.147537,0.147537,0.147537


In [ ]:
# inserting in db
TABLE = "agg_soil_moisture"
queryHandler = QueryHandler()
queryHandler.createAggSoilMoistureTableReq(db)

In [ ]:
sm_df.to_sql(TABLE, conn, schema="public", if_exists="replace")